In [12]:
import math
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.integrate import quad

In [13]:
#BSM模型定价
def dN(x):
    ''' Probability density function of standard normal random variable x. '''
    return math.exp(-0.5 * x ** 2) / math.sqrt(2 * math.pi)

def N(d):
    ''' Cumulative density function of standard normal random variable x. '''
    return quad(lambda x: dN(x), -20, d, limit=50)[0]

def d1f(St, K, t, T, r, div,sigma):
    ''' Black-Scholes-Merton d1 function.
        Parameters see e.g. BSM_call_value function. '''
    d_up = math.log(St / K) + ((r - div + 0.5 * sigma ** 2)*(T - t))
    d1 = d_up / (sigma * math.sqrt(T - t))
    return d1
#
# Valuation Functions
#
def BSM_call_value_descrpition(St, K, t, T, r, div,sigma):
    ''' Calculates Black-Scholes-Merton European call option value.
    Parameters
    ==========
    St : float
        stock/index level at time t
    K : float
        strike price
    t : float
        valuation date
    T : float
        date of maturity/time-to-maturity if t = 0; T > t
    r : float
        constant, risk-less short rate
    div : float
        dividend
    sigma : float
        volatility
    Returns
    =======
    call_value : float
        European call present value at t
    '''
    d1 = d1f(St, K, t, T, r, div,sigma)
    d2 = d1 - sigma * math.sqrt(T - t)
    list = [d1,d2]
    
    return list
def BSM_call_value_new(St, K, t, T, r, div,sigma,Nd1,Nd2):
    ''' Calculates Black-Scholes-Merton European call option value.
    Parameters
    ==========
    St : float
        stock/index level at time t
    K : float
        strike price
    t : float
        valuation date
    T : float
        date of maturity/time-to-maturity if t = 0; T > t
    r : float
        constant, risk-less short rate
    div : float
        dividend
    sigma : float
        volatility
    Returns
    =======
    call_value : float
        European call present value at t
    '''
    call_value = St *math.exp(-div * (T - t)) * Nd1 - math.exp(-r * (T - t)) * K * Nd2
    return call_value
def BSM_put_value_new(St, K, t, T, r, div,sigma,Nd3,Nd4):
    ''' Calculates Black-Scholes-Merton European call option value.
    Parameters
    ==========
    St : float
        stock/index level at time t
    K : float
        strike price
    t : float
        valuation date
    T : float
        date of maturity/time-to-maturity if t = 0; T > t
    r : float
        constant, risk-less short rate
    div : float
        dividend
    sigma : float
        volatility
    Returns
    =======
    call_value : float
        European call present value at t
    '''
    put_value = math.exp(-r * (T - t)) * K * Nd4 - St *math.exp(-div * (T - t)) * Nd3 
    return put_value

#插值法计算N（d1,d2）
def nd_interpolation(high,low,dd,high_p,low_p):
    ''' 
    high: float 比d1进一位
    low: float 比d1退一位
    dd :float 上述已求出的
    high_p:查表出来概率 low_p 同
    '''
    value = (high-dd)*low_p/(high - low)+(dd-low)*high_p/(high - low)
    return value
#省略四舍五入
def cut(num, c):
    c=10**(-c)
    return (num//c)*c

# BSM利用插值法计算价格

In [26]:
# 输入参数
S0_BSM = 40  # index level
K = 40  # option strike
T = 0.5  # maturity date
r = 0.05  # risk-less short rate
div=0.05 # dividend yield 
sigma = 0.3  # volatility

In [27]:
## Step 1:Call期权价格过程结果详解求 d1 d2
print(BSM_call_value_descrpition(S0_BSM, K,0 , T, r, div,sigma))

[0.10606601717798213, -0.10606601717798213]


### ！！！计算步骤2 (d1)
1. 带入d1
2. 查表上下p

In [28]:
## Step 2:利用 d1 d2求Nd1Nd2
### 插值法输入Nd1参数
dd = cut(BSM_call_value_descrpition(S0_BSM, K,0 , T, r, div,sigma)[0],4)#保留四位         
low = cut(dd,2) #保留两位
high_old =cut(dd,2)+0.01 #保留两位
high = round(high_old,2)
print('high =',high,'---> 查表求分布')
print('low =',low,'---> 查表求分布')
high_p = round(N(high),4) #查表
low_p = round(N(low),4) #查表
# high_p = 0.6064 #查表
#low_p = 0.6026 #查表

high = 0.11 ---> 查表求分布
low = 0.1 ---> 查表求分布


### ！！！计算步骤3 （d2）
1. 带入d2
2. 查表上下p

In [29]:
# 同理插值法输入Nd2参数
dd2 =cut(BSM_call_value_descrpition(S0_BSM, K,0 , T, r, div,sigma)[1],4) #保留四位
high_old2 =cut(dd2,2)+0.01 #保留两位
high2 = round(high_old2,2)
low2 = cut(dd2,2) 
print('high =',high2,'---> 查表求分布')
print('low =',low2,'---> 查表求分布')
## high_p2 = round(N(high2),4) #查表
## low_p2 = round(N(low2),4) #查表
high_p2 = 0.6179 #查表
low_p2 = 0.6141 #查表

high = -0.1 ---> 查表求分布
low = -0.11 ---> 查表求分布


In [30]:
'''print('N(d1)原始',nd_interpolation(high,low,dd,high_p,low_p))
print('保留四位后',cut(nd_interpolation(high,low,dd,high_p,low_p),4))
'''

# 计算Nd1
Nd1 = round(nd_interpolation(high,low,dd,high_p,low_p),4)
##如果二者不想等不对使用下面
##Nd2= round(nd_interpolation(high2,low2,dd2,high_p2,low_p2),4)-1

Nd3 = round((1- Nd1),4)
print('N(d1) = ',Nd1)
print('N(-d1) = ',Nd3)

N(d1) =  0.5422
N(-d1) =  0.4578


In [31]:
##print('Nd(2)=',nd_interpolation(high2,low2,dd2,high_p2,low_p2))
#print('保留四位后',round(nd_interpolation(high2,low2,dd2,high_p2,low_p2),4))

Nd2= round(nd_interpolation(high2,low2,dd2,high_p2,low_p2),4)
Nd4 = 1 - Nd2 ## N(-d2)  
if Nd2+Nd4!=1:
    Nd4 +=0.0001
print('N(d2) = ',Nd2)
print('N(-d2) = ',Nd4)

N(d2) =  0.6156
N(-d2) =  0.38439999999999996


In [32]:
## Call期权价格结果 (应试版)
answer= BSM_call_value_new(S0_BSM, K,0 , T, r, div,sigma,Nd1,Nd2)
print("Vc = ",answer)

Vc =  -2.863509901715183


## 计算put

In [33]:
## 期权价格结果
put_answer = round(BSM_put_value_new(S0_BSM, K,0 , T, r, div,sigma,Nd3,Nd4),4)
print("Vp = ",put_answer)


Vp =  -2.8635


In [34]:
###答案填空
print('d1:',BSM_call_value_descrpition(S0_BSM, K,0 , T, r, div,sigma)[0])
print('d2:',BSM_call_value_descrpition(S0_BSM, K,0 , T, r, div,sigma)[1])
print('=============')
print('d1_high =',high,'')
print('d1_low =',low,'')
print('d2_high2 =',high2,'')
print('d2_low2 =',low2,'')
print('=============')
print('N(d1) = ',Nd1)
print('N(d2) = ',Nd2)
print('N(-d1) = ',Nd3)
print('N(-d2) = ',Nd4)
print('=============')
print("Call_option Vc = ",answer)
print("put_option Vc = ",put_answer)

d1: 0.10606601717798213
d2: -0.10606601717798213
d1_high = 0.11 
d1_low = 0.1 
d2_high2 = -0.1 
d2_low2 = -0.11 
N(d1) =  0.5422
N(d2) =  0.6156
N(-d1) =  0.4578
N(-d2) =  0.38439999999999996
Call_option Vc =  -2.863509901715183
put_option Vc =  -2.8635
